In [7]:
import tensorflow as tf
import numpy as np
import keras
import os

In [8]:
monks1_train_path = os.getcwd() + '/../../data/monks/monks-1.train'
data = np.loadtxt(monks1_train_path, usecols=range(8))
data

ValueError: could not convert string 'data_5' to float64 at row 0, column 8.